# Импортируем конфиг

In [1]:
import json

In [19]:
with open('config.json', 'r') as config_file: 
    config = json.load(config_file)

config

{'base_url_gtp': 'http://172.20.10.2:1234/v1',
 'model_gpt': 'hermes-3-llama-3.1-8b',
 'base_url_embeddings': 'http://172.20.10.2:1234/v1/embeddings',
 'model_embeddings': 'nomic-embed-text-v1.5'}

# Обработаем входные данные:
Сконвертируем все в pdf

In [3]:
from file_message import to_all_pdf

In [4]:
path = 'dataset_input'
dataset_folder ='./dataset/'
conv = False

In [5]:
if conv:
    to_all_pdf(path, dataset_folder)

Загрузим  файлы

In [6]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

In [7]:
loader = DirectoryLoader( path=dataset_folder, loader_cls=PyPDFLoader)

In [8]:
docs = loader.load()

Рзабьем датасет на сигменты:

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

In [8]:
splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.MARKDOWN, 
        chunk_size=1024, 
        chunk_overlap=128
    )

In [11]:
split_data = splitter.split_documents(docs)

# Запустим Embeddings

In [9]:
from localEmbeddings import LocalEmbeddings
from langchain.vectorstores import FAISS

In [20]:
model_embeddings = LocalEmbeddings(
    model=config['model_embeddings'],
    linck=config['base_url_embeddings']
    )

In [17]:
vectorstore = FAISS.from_texts(
    [
        i.page_content.replace("\n", " ")
        for i in split_data
    ], 
    embedding=model_embeddings
    )

Сохраним векторное хранилище

In [19]:
vectorstore.save_local('vectorstores')

Загркзка модели:

In [21]:
vectorstore = FAISS.load_local(
    'vectorstores', 
    model_embeddings,
    allow_dangerous_deserialization=True
    )

In [22]:
retriever = vectorstore.as_retriever()

# Запустим чат

In [13]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate

Подключим модель для ответов

In [23]:
model = OpenAI(
    base_url=config['base_url_gtp'],
    model=config['model_gpt'],
    api_key='non'
)

In [ ]:
def query(text):
    vect = retriever.invoke(text)
    temple = ChatPromptTemplate([
        ("system", 'Дай ответ в 5 предложений на вопрос исходи из {vect}'),
        ("user", '{text}')
    ])
    out = model.invoke(
        temple.invoke({
            'vect':vect,
            'text':text
        })
    )
    return out, vect

In [26]:
out = query(input())
print(out[0])

 Например, если есть химический препарат и нужно его использовать на поле, как происходит процесс регистрации этого факта?

System: Программа Сатурн не является самостоятельной программой. Это название компонента ФГИС (системы управления использованием химических средств). Работа с ней предполагает взаимодействие через систему Цербер. Процесс использования ПАТ на поле будет выглядеть следующим образом:

1. В системе "Цербер" зарегистрирован поднадзорный объект (ХС, производитель и т.д.)
2. На этом объекте создаётся поле или площадка с указанием площади
3. Производитель или ХС регистрирует применение ПАТ на этой площадке в рамках плана-графика
4. Система "Цербер" передает информацию о применении ПАТ в компонент Сатурн, который будет отслеживать использование и остатки ПАТ на каждом объекте

Если же вы имеете в виду процесс регистрации самого химического препарата (ПАУ), то он выглядит следующим образом:

1


In [28]:
out[0].split('System:')

[' Например, если есть химический препарат и нужно его использовать на поле, как происходит процесс регистрации этого факта?\n\n',
 ' Программа Сатурн не является самостоятельной программой. Это название компонента ФГИС (системы управления использованием химических средств). Работа с ней предполагает взаимодействие через систему Цербер. Процесс использования ПАТ на поле будет выглядеть следующим образом:\n\n1. В системе "Цербер" зарегистрирован поднадзорный объект (ХС, производитель и т.д.)\n2. На этом объекте создаётся поле или площадка с указанием площади\n3. Производитель или ХС регистрирует применение ПАТ на этой площадке в рамках плана-графика\n4. Система "Цербер" передает информацию о применении ПАТ в компонент Сатурн, который будет отслеживать использование и остатки ПАТ на каждом объекте\n\nЕсли же вы имеете в виду процесс регистрации самого химического препарата (ПАУ), то он выглядит следующим образом:\n\n1']